### TP1 d'Apprentissage Statistique 

### A- Les données 

La fonction qui permet d'engendrer une matrice donc chaque valeur est une observation d'une personne avec 3 caractéristiques 

In [2]:
import random
import numpy as np 

def generer_tableau(n): # n est le nombre de lignes de la matrice, le nombre de colonnes est 3

    # Nous définissons les domaines de chaque observation, dans lesquels nous allons faire la sélection des valeurs 
    # pour former la matrice 

    cheveux = ['B', 'D']   # Blond, Dark
    hauteur = ['T', 'S']   # Tall, Short
    pays = ['G', 'P']      # Greenland, Poland
    
    tableau = []
    for _ in range(n):  # pour chaque ligne
        ligne = []
        for _ in range(3):  # 3 colonnes
            c = random.choice(cheveux)
            h = random.choice(hauteur)
            p = random.choice(pays)
            ligne.append((c, h, p))  # un triplet
        tableau.append(ligne)
    return tableau

# Générons une matrice un peu plus grande avec 100 lignes 
print(generer_tableau(100))


[[('D', 'S', 'P'), ('D', 'S', 'P'), ('B', 'S', 'P')], [('B', 'S', 'G'), ('B', 'T', 'P'), ('D', 'S', 'G')], [('D', 'S', 'G'), ('B', 'S', 'G'), ('B', 'T', 'G')], [('D', 'S', 'G'), ('D', 'T', 'P'), ('B', 'S', 'P')], [('D', 'T', 'G'), ('B', 'S', 'P'), ('B', 'T', 'G')], [('D', 'T', 'P'), ('B', 'T', 'P'), ('D', 'T', 'P')], [('B', 'T', 'P'), ('B', 'S', 'P'), ('B', 'T', 'G')], [('D', 'S', 'P'), ('B', 'T', 'G'), ('D', 'T', 'G')], [('D', 'S', 'G'), ('B', 'T', 'P'), ('D', 'S', 'G')], [('D', 'T', 'P'), ('D', 'T', 'P'), ('B', 'T', 'P')], [('D', 'S', 'G'), ('B', 'S', 'G'), ('B', 'T', 'G')], [('D', 'T', 'P'), ('D', 'S', 'P'), ('B', 'T', 'G')], [('B', 'S', 'P'), ('D', 'T', 'G'), ('B', 'T', 'P')], [('B', 'S', 'G'), ('D', 'S', 'P'), ('B', 'S', 'P')], [('D', 'T', 'P'), ('D', 'S', 'G'), ('D', 'T', 'G')], [('B', 'S', 'G'), ('B', 'S', 'P'), ('B', 'T', 'G')], [('B', 'S', 'G'), ('D', 'S', 'G'), ('D', 'S', 'P')], [('B', 'S', 'G'), ('D', 'T', 'G'), ('B', 'T', 'P')], [('B', 'S', 'P'), ('B', 'S', 'P'), ('B', 'S',

In [3]:
# La matrice que nous allons utiliser 

# Pour controler l'aléa 
random.seed(42)
data = generer_tableau(100)

### B- Questions : Origine le plus probable d'un individu grand (T) et qui a les cheveux Blond (B)

Les fonctions que nous allons écrire dans la suite sont par rapport à la matrice que nous avons définie

### C- Maximum a posteriori
Nous allons utiliser deux versions de cette métrique pour conclure sur la question:  
- Une qui se fera avec naive bayes 
- Une autre qui ne tiendra pas compte de navive bayes : se basera uniquement sur les fréquences d'apparution, via la formule des probabilités conditionnelles

In [4]:
## Nous allons donc faire le calcul de probabilité avec plusieurs petites fonctions que nous allons utiliser
# par la suite dans une fonction générale 

##1- Version avec le théorème de bayes

"""
Il s'agit de calculer deux probabilités a posteriori et de prendre le maximum afin de conclure sur la nationalité de l'individu
Les formules sont les suivantes: 

P(G/B,T) = P(G)*P(B,T/G) / P(B,T)
P(P/B,T) = P(P)*P(B,T/P) / P(B,T)

Nous aurons besoin de caluler :
P(G): pg(), 
P(P): pp(), 
P(B,T/G): pbtg(), 
P(B,T/P): pbtp() 
et P(B,T): pbt()

P(G) = nombre de personnes de nationalité G / nombre total d'individus
P(P) = nombre de personnes de nationalité P / nombre total d'individus

P(B,T/G) = P(B/G)*P(T/G) via l'hypothèse d'indépendance de naive bayes
P(B/G) : pbg() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité G
P(T/G) : ptg() = nombre de personnes de hauteur T / nombre de personnes de nationalité G

P(B,T/P) = P(B/P)*P(T/P) pareil que précédemment pour la nationalité G
P(B/P) : pbp() = nombre de personnes aux cheveux Blonds / nombre de personnes de nationalité P
P(T/P) : ptp() = nombre de personnes de hauteur T / nombre de personnes de nationalité P

"""


# -------------- Fonctions utilitaires -------------------- #


def pg(data):
    nb_G = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
    return nb_G / nb_Total

def pp(data):
    nb_P = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
    return nb_P / nb_Total

def pbg(data):
    nb_blond_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_G += 1
    return nb_blond_G / nb_G if nb_G > 0 else 0

def ptg(data):
    nb_T_G, nb_G = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'G':
                nb_G += 1
                if indiv[1] == 'T':
                    nb_T_G += 1
    return nb_T_G / nb_G if nb_G > 0 else 0

def pbp(data):
    nb_blond_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[0] == 'B':  # cheveux blonds
                    nb_blond_P += 1
    return nb_blond_P / nb_P if nb_P > 0 else 0

def ptp(data):
    nb_T_P, nb_P = 0, 0
    for ligne in data:
        for indiv in ligne:
            if indiv[2] == 'P':
                nb_P += 1
                if indiv[1] == 'T':
                    nb_T_P += 1
    return nb_T_P / nb_P if nb_P > 0 else 0

def pbtg(data):
    return pbg(data) * ptg(data)

def pbtp(data):
    return pbp(data) * ptp(data)

def pbt(data):
    # P(B,T) = somme des cas pour G et P
    # Nous appliquons la loi des proba totales, vu qu'on veut les individus 
    # avec des cheveux B et une hauteur T quelque soit la nationalité
    
    return pbtg(data) * pg(data) + pbtp(data) * pp(data)


# -------------------------Fonction générale------------------#


def classer_individu(data):
    # probas a posteriori
    p_G = (pg(data) * pbtg(data)) / pbt(data)
    p_P = (pp(data) * pbtp(data)) / pbt(data)

    if p_G > p_P:
        return f"Nationalité G avec une proba Max de {p_G}"
    elif p_P > p_G:
        return f"Nationalité P avec une proba Max de {p_P}"
    else:
        return "Indécis (égalité)"


In [5]:
classer_individu(data)

'Nationalité P avec une proba Max de 0.510296618174948'

### Conclusion sur la nationalité avec la première approche: 

Nous voyons que la probabilité de la nationalité P est celle trouvé comme plus grande 

## Reprenons la fonction cette fois sans utiliser le théorème de naive bayes

Il sera question maintenant de faire le calcul de proba conditionnelle

P(G/B,T) = P(B,T,G)/P(B,T) un peu comme la proba de l'intersection de G avec (B,T) divisé par P(B,T)

Ici il s'agit de compter le nombre de fois qu'on a un individu B, T, G et diviser par le nombre d'individu total ensuit diviser par le calul du nombre de personnes aux cheveux B et une hauteur T et diviser par le nombre total d'individus 

P(P/B,T): on fait pareil sur la nationalité P, on regarde le max et on conclu sur la nationalité 

In [ ]:
# 2- Version qui n'utilise pas le théorème de Bayes 

"""
Ici, on calcule directement les probabilités par comptage dans le jeu de données.

Formules (probabilité conditionnelle par fréquence):

P(G / B,T) = P(B,T,G) / P(B,T) 
P(B,T,G) = nombre d'individus (B,T,G) / nombre total d'individus
P(B,T)   = nombre d'individus (B,T,*) / nombre total d'individus

P(P / B,T) = P(B,T,P) / P(B,T) 
P(B,T,P) = nombre d'individus (B,T,P) / nombre total d'individus
"""

#-------- Fonctions utilitaires ------------

def pbtg_v2(data):
    nb_BTG = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'G':
                nb_BTG += 1
    return nb_BTG / nb_Total if nb_Total > 0 else 0

def pbtp_v2(data):
    nb_BTP = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T' and indiv[2] == 'P':
                nb_BTP += 1
    return nb_BTP / nb_Total if nb_Total > 0 else 0

def pbt_v2(data):
    nb_BT = 0
    nb_Total = len(data) * len(data[0])
    for ligne in data:
        for indiv in ligne: 
            if indiv[0] == 'B' and indiv[1] == 'T':  # Nationalité indifférente
                nb_BT += 1
    return nb_BT / nb_Total if nb_Total > 0 else 0

def classer_individu_v2(data):
    p_G = pbtg_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    p_P = pbtp_v2(data) / pbt_v2(data) if pbt_v2(data) > 0 else 0
    
    if p_G > p_P:
        return f"Nationalité G avec proba:  (p={p_G:.3f})"
    elif p_P > p_G:
        return f"Nationalité P avec proba: (p={p_P:.3f})"
    else:
        return f"Indécis (égalité, la proba est: p={p_G:.3f})"


In [7]:
classer_individu_v2(data)

'Nationalité P avec proba: (p=0.560)'

### Commentaire:

À partir de notre jeu de données, on observe que les deux approches attribuent à l’individu la nationalité P. Toutefois, la seconde approche semble produire une probabilité légèrement plus élevée.

La principale différence entre les deux approches réside dans la stabilité des résultats : la première approche, qui repose sur l’hypothèse d’indépendance conditionnelle entre les variables, est moins sujette à des probabilités nulles. En effet, elle utilise la décomposition P(B/G) × P(T/G), ce qui permet de contourner les cas rares ou absents dans les données.

À l’inverse, la seconde approche calcule directement P(B, T, G), ce qui peut poser problème lorsque cette combinaison spécifique est peu représentée dans le jeu de données. Cela peut entraîner des probabilités nulles ou très faibles, rendant l’approche moins robuste dans des contextes de données clairsemées.


## D- Estimateur de Maximum de Vraisemblance : MLE

Il est question dans cette partie de trouver maintenant la classe qui maximise plutôt cette probabilité : P((B,T)/nationalité)

Donc *P(observation/classe)*:
- P((B,T)/P) : la nationalité P
- P((B,T)/G) : la nationalité G

Nous devons faire ceci en utilisant deux approches:
1. En tenant compte du cadre bayes naïf
P(B,T/G) = P(B/G)*P(T/G) comme donné dans le TP pareil pour P(B,T/P)
2. Sans tenir compte du cadre bayes naïf

    Ici on fonctionnera par comptage dans notre matrice de données, comme avec l'exemple donné dans le TP 


In [12]:
# Le code pour répondre à la question avec la première approche est le suivant 


# Notre fonction est un peu plus générale, nous allons lui passer les données, la couleur des cheveux B et la hauteur T pour voir 
# la classe qui maximise la vraisemblance 
def mle_naif(data, cheveux, hauteur):
    # Comptage des individus par classe
    # On récupère de manière individuelle les classes présentes dans les données
    classes = set(indiv[2] for ligne in data for indiv in ligne)

    # où on pourra stocker les vraissemblances calculées et retenir la valeur maximale 
    vraisemblances = {}

    # rappel: le triplet (B,T,G) est dans l'ordre des caractéristiques (cheveux, hauteur et nationalité)
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_cheveux_c = sum(indiv[2] == c and indiv[0] == cheveux for ligne in data for indiv in ligne)
        nb_hauteur_c = sum(indiv[2] == c and indiv[1] == hauteur for ligne in data for indiv in ligne)
        
        p_cheveux = nb_cheveux_c / nb_c if nb_c > 0 else 0
        p_hauteur  = nb_hauteur_c / nb_c if nb_c > 0 else 0
        
        vraisemblances[c] = p_cheveux * p_hauteur
    
    # Choisir la classe qui maximise P(x|c)
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [ ]:
## Test de notre fonction pour répondre à la question

print(f"La distribution de vraisemblance entre les classes est: {mle_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution de vraisemblance entre les classes est: {'G': 0.24319759804841432, 'P': 0.24025974025974026} 
La nationalité finale est: G


In [17]:
# Le code pour répondre à la question avec la deuxième approche est le suivant

def mle_sans_naif(data, cheveux, hauteur):
    classes = set(indiv[2] for ligne in data for indiv in ligne)
    vraisemblances = {}
    
    for c in classes:
        nb_c = sum(indiv[2] == c for ligne in data for indiv in ligne)
        nb_obs_c = sum(indiv[2] == c and indiv[0] == cheveux and indiv[1] == hauteur 
                       for ligne in data for indiv in ligne)
        
        vraisemblances[c] = nb_obs_c / nb_c if nb_c > 0 else 0
    
    best_class = max(vraisemblances, key=vraisemblances.get)
    return best_class, vraisemblances


In [18]:
# Application de la fonction avec le cas (B,T,G)

print(f"La distribution des proba entre les classes est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[1]} \nLa nationalité finale est: {mle_sans_naif(data=data, cheveux='B', hauteur='T')[0]}")

La distribution des proba entre les classes est: {'G': 0.22602739726027396, 'P': 0.2727272727272727} 
La nationalité finale est: P
